<a href="https://colab.research.google.com/github/LapTQ/image_captioning/blob/main/image_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/hungpham13/Vietnamese-HTR.git

Cloning into 'Vietnamese-HTR'...
remote: Enumerating objects: 2403, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2403 (delta 0), reused 4 (delta 0), pack-reused 2399
Receiving objects: 100% (2403/2403), 427.59 MiB | 42.71 MiB/s, done.
Checking out files: 100% (2395/2395), done.


In [1]:
import os
import re
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from tensorflow import keras
from pathlib import Path

seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

In [6]:
!head -20 '/content/Vietnamese-HTR/Data 1: Handwriting OCR for Vietnamese Address/0825_DataSamples 1/labels.json'

{
    "1.jpg": "Số 3 Nguyễn Ngọc Vũ, Hà Nội",
    "2.jpg": "Số 30 Nguyên Hồng, Láng Hạ, Đống Đa, Hà Nội",
    "3.jpg": "58 Thái Thịnh, Đống Đa, Hà Nội",
    "4.jpeg": "Số 370/8 khu phố 5B, phường Tân Biên, Biên Hòa, Đồng Nai",
    "5.jpg": "Vĩnh Trung Plaza, B, 255-257 đường Hùng Vương, phường Vĩnh Trung",
    "6.jpg": "Tòa nhà 34T, Hoàng Đạo Thúy, Hà Nội",
    "7.jpg": "40 Cát Linh, Đống Đa, Hà Nội",
    "8.jpg": "phòng 101, tầng 1, lô 04-TT5B, khu đô thị Tây Nam Linh Đàm",
    "9.JPG": "Nhà 87 ngõ 416 Đê La Thành",
    "10.JPG": "Up coworking Space, 89 Láng Hạ, Hà Nội",
    "11.jpg": "192 Ngô Đức Kế, quận 1, Hồ Chí Minh",
    "12.jpg": "số 5 Công Trường Mê Linh, phường Bến Nghé, quận 1",
    "13.jpg": "90A đường Mai Xuân Thưởng, tỉnh Gia Lai",
    "14.jpg": "96/7/12B Phạm Văn Đồng, thành phố Pleiku",
    "15.jpg": "168 Ngô Gia Tự, thành phố Hà Tĩnh"
}

In [36]:
train_img_dir = '/content/Vietnamese-HTR/Data 1: Handwriting OCR for Vietnamese Address/0916_Data Samples 2'
test_img_dir = '/content/Vietnamese-HTR/Data 1: Handwriting OCR for Vietnamese Address/1015_Private Test'

image_height, image_width = 120, 1900
vocab_size = 10000
seq_length = 25
embed_dim = 512
ff_dim = 512

batch_size = 54
epochs = 30
AUTOTUNE = tf.data.AUTOTUNE

In [35]:
print('Number of training images:', len(list(Path(train_img_dir).glob('*.png'))))
print('Number of testing images:', len(list(Path(test_img_dir).glob('*.png'))))

Number of training images: 1823
Number of testing images: 549


In [41]:
import json

train_json = json.load(
    open(train_img_dir + '/labels.json', 'r')
)

test_json = json.load(
    open(test_img_dir + '/labels.json', 'r')
)

In [20]:
# max([len(label.split()) for label in json_file.values()])
# tokens = set()
# for label in json_file.values():
#     for token in label.split():
#         tokens.add(token)
# len(tokens)

2494

In [43]:
def decode_and_resize(img_path):
    img_string = tf.io.read_file(img_path)
    img = tf.image.decode_png(img_string)

    # resize to desired shape
    img = tf.image.resize_with_pad(img, image_height, image_width)

    # This will convert to float values in [0, 1]
    # img = tf.image.convert_image_dtype(img, tf.float32)

    return img

vectorization = keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_length,
    standardize=lambda label: tf.strings.regex_replace(label, "[%s]" % re.escape("!\"#$%&'()*+,.:;<=>?@[\]^_`{|}~"), "")
)

vectorization.adapt(list(train_json.values()))

def preprocess_input(img_path, label):
    return decode_and_resize(img_path), vectorization(label)

def make_dataset(img_paths, labels, training):
    assert training is True or training is False

    dataset = tf.data.Dataset.from_tensor_slices((img_paths, labels))
    dataset = dataset.map(preprocess_input, num_parallel_calls=AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=2000)
    dataset = dataset.cache()
    if training: 
        dataset = dataset.shuffle(buffer_size=2000)
    dataset = dataset.batch(batch_size)


train_ds = make_dataset(
    [str(path) for path in sorted(list(Path(train_img_dir).glob('*.png')))],
    train_json.values(),
    training = True
)
test_ds = make_dataset(
    [str(path) for path in sorted(list(Path(test_img_dir).glob('*.png')))],
    test_json.values(),
    training = False
)

In [ ]:
# for label in json_file.values():
#     print(
#         tf.strings.regex_replace(label, "[%s]" % re.escape("!\"#$%&'()*+,.:;<=>?@[\]^_`{|}~"), "")
#     )

In [31]:
print(list(json_file.values())[0])
vectorization(list(json_file.values())[0])

Số 253 đường Trần Phú, Thị trấn Nam Sách, Huyện Nam Sách, Hải Dương


<tf.Tensor: shape=(25,), dtype=int64, numpy=
array([ 11, 875,  46,  75,  20,  18,  39,  37, 369,   4,  37, 369,  32,
        52,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])>

## References
1. https://arxiv.org/pdf/1703.09137.pdf
2. https://viblo.asia/p/a-guide-to-image-captioning-part-1-gioi-thieu-bai-toan-sinh-mo-ta-cho-anh-gAm5yr88Kdb
3. https://www.tensorflow.org/tutorials/text/image_captioning
4. https://arxiv.org/pdf/1502.03044.pdf
5. https://keras.io/examples/vision/image_captioning/
